<a href="https://colab.research.google.com/github/Santiago8888/Kin/blob/master/Job_s_PCA_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
cd gdrive/My Drive/Dzeta

In [0]:
import json
with open('Data/angel_V1.json') as json_file:
    data = json.load(json_file)

In [0]:
import pandas as pd
df = pd.DataFrame.from_records(data[1:], columns=data[0])

In [0]:
descriptions = df['description'].tolist()
descriptions = sum(descriptions, [])

In [0]:
import nltk
import torch
import pandas as pd

import numpy as np
from sklearn.cluster import KMeans

!pip install nmslib 
import nmslib

nltk.download('punkt')

In [0]:
from InferSent.models import InferSent
model_version = 2
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048, 'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}

model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)
model.build_vocab(descriptions, tokenize=True)
embeddings = model.encode(sentences=descriptions, tokenize=True)


In [0]:
"""
My Dataset:



"""